# Description

Script for extracting the rotation matrix using the real joint angles and publishing it to a ROS topic

# Imports

In [1]:
import os
import gym
import numpy
import pandas
import mujoco_py

import rospy
from sensor_msgs.msg import JointState
from std_msgs.msg import Float64MultiArray

# Paths

In [2]:
HOME = os.getenv("HOME")
MODEL_PATH = os.path.join(*[HOME, "DRL_AI4RoMoCo", "code", "environment", "UR10", "ur10_heg.xml"])

# Configuration

In [3]:
q_init = numpy.array([0, -1.3, 1.0, -0.80, 1.5708, 0.0])

# Functions

In [4]:
def set_state(qpos):
    assert qpos.shape == (model.nq,)
    old_state = sim.get_state()
    new_state = mujoco_py.MjSimState(old_state.time, qpos, old_state.qvel,
                                     old_state.act, old_state.udd_state)
    sim.set_state(new_state)
    sim.forward()

In [5]:
def q_callback(data):
    '''Callback function for receiving Joint States'''
    global q
    q = numpy.array(data.position)[[2,1,0,3,4,5]]
    set_state(q)
    xmat = sim.data.get_body_xmat("gripper_dummy_heg")
    publish_xmat(xmat)

In [6]:
vector = numpy.array([1,2,3])

In [7]:
def publish_xmat(xmat):
    '''Message which includes the rotation matrix'''

    message = Float64MultiArray()
    message.data = xmat.reshape((9,))
    xmat_publisher.publish(message)

# Main

In [8]:
model = mujoco_py.load_model_from_path(MODEL_PATH)
sim = mujoco_py.MjSim(model)
set_state(q_init)

In [9]:
rospy.init_node("forward_kinematics", anonymous=True)
rospy.Subscriber("/joint_states", JointState, q_callback)
xmat_publisher = rospy.Publisher("/rotation_matrix", Float64MultiArray, queue_size=1)

In [ ]:
while True:
    rospy.spin()